In [1]:
import pandas as pd
import numpy as np
import holidays
import datetime
import warnings
warnings.filterwarnings("ignore")

bloki_cene_moci = np.array([0.24923, 0.04877, 0.01103, 0.00038, 0.00000]) + np.array([3.36401, 0.83363, 0.18034, 0.01278, 0.00000])
bloki_cene_energije = np.array([0.00663, 0.00620, 0.00589, 0.00592, 0.00589]) + np.array([0.01295, 0.01224, 0.01248, 0.01246, 0.01258])
# penali = 

prikljucna_moc = 14
skupina_koncnih_odjemalcev = 0
st_faz = 3
net_metering = False
Fex = 0.9  # faktor presežene obračunske moči
VAT = 1.22

In [2]:

def energija_omreznine(df, bloki_cene_energije, placilo_omr_e):
    cene_e = placilo_omr_e
    df_energy = df.loc[df.a > 0]
    df_energy = df_energy[["a", "year_month", "block"]].groupby(["year_month", "block"]).sum()
    for year_month, vals in df_energy.groupby(level=0):
        _, month = year_month.split("-")
        df = vals.droplevel(0)
        # df = vals
        for i in range(1, 6):
            if i not in df.index:
                # insert it at the beginning
                df.loc[i] = 0
        cena = df["a"] * bloki_cene_energije
        cene_e[int(month) - 1] = cena.sum()
    cene_e = np.array(cene_e)
    return cene_e

def moc_omreznine(df, bloki, bloki_cene_moci, placilo_omr_p):
    placilo_p = placilo_omr_p
    power = df[["p", "year_month", "block"]].groupby(["year_month", "block"]).max()
    for year_month, _ in power.groupby(level=0):
        _, month = year_month.split("-")
        if int(month) in [1,2,11,12]:
            cena = bloki * bloki_cene_moci * [1, 1, 1, 1, 0]
        else:
            cena = bloki * bloki_cene_moci * [0, 1, 1, 1, 1]
        placilo_p[int(month) - 1] = cena.sum()

    placilo_p = np.array(placilo_p)
    return placilo_p

from utils import handle_obr_moc, find_min_obr_p

def predlagane_obr_moci(df, prikljucna_moc, st_faz, skupina_koncnih_odjemalcev):
    blocks = []
    for block in range(1, 6):
        if skupina_koncnih_odjemalcev <= 1:
            selected_season = df[df.month.isin([1,2,11,12])]
            block_value = np.average(selected_season.loc[selected_season.block == block, "p"].nlargest(3))
        else:
            block_value = np.average(df.loc[df.block == block, "p"].nlargest(1))
        blocks.append(block_value)
    min_obr_p = find_min_obr_p(st_faz, prikljucna_moc)
    blocks = np.array(handle_obr_moc(blocks, prikljucna_moc, min_obr_p))
    # round
    blocks = np.round(blocks, 1)
    return blocks

def penali(df, bloki, Fex, placilo_omr_penali):
    cene_penali = placilo_omr_penali
    df["block_max_power"] = df["block"].apply(lambda x: bloki[x-1] if x < len(bloki) else None)

    # subtract the block_max_power from actual power p and only take positive values.
    diff = df["p"] - df["block_max_power"]
    df["power_exceeded"] = diff * (diff > 0)
    df = df[["year_month", "block", "power_exceeded"]]
    df_penali = df.groupby(["block", "year_month"]).sum()

    for _, vals in df_penali.groupby(level=0):
        df = vals.droplevel(1)
        cena = Fex * np.sqrt(sum(df["power_exceeded"]**2))
        block = df.index[0]
        cene_penali[block - 1] = cena
    cene_penali = np.array(cene_penali)
    return cene_penali

In [3]:
def individual_tariff_times(dates: np.array) -> np.array:
    """
    Generates block assignments (1-5) for the given dates.

    Takes an array of dates and returns an array of block assignments (1-5).

    Args:
    ----------
        dates: np.array
            Array of dates in format of Pandas datetime

    Returns:
    ----------
        block_assignments: np.array
            Array where each entry corresponds to the block number (1-5) for the given date.
    """
    # Prepare array of holidays
    si_holidays = holidays.SI(years=(dates.iloc[0].year, dates.iloc[-1].year))

    # Tariff block values (obr_p_values table)
    high_season_working = [
        3, 3, 3, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 3, 3
    ]
    low_season_working = [
        4, 4, 4, 4, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 3, 4, 4
    ]
    high_season_workoff = [
        4, 4, 4, 4, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 3, 4, 4
    ]
    low_season_workoff = [
        5, 5, 5, 5, 5, 5, 4, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 5, 5
    ]

    # Combine the seasons
    high_season = [high_season_working, high_season_workoff]
    low_season = [low_season_working, low_season_workoff]
    obr_p_values = [low_season, high_season]

    # Initialize array for block assignments
    block_assignments = np.zeros(len(dates), dtype=int)

    # Loop through dates to assign blocks
    for i, date in enumerate(dates):
        # Adjust for 15 minutes earlier (tariff is for the previous quarter-hour)
        date -= datetime.timedelta(minutes=15)

        # Determine if it's a holiday or weekend
        workoff = 1 if date in si_holidays or date.weekday() > 4 else 0

        # Determine if it's high season (November to February)
        high_season_flag = 1 if date.month in [1, 2, 11, 12] else 0

        # Get the hour
        hour = date.hour

        # Determine the block (subtract 1 for 0-based indexing)
        block = obr_p_values[high_season_flag][workoff][hour]

        # Assign the block number to the output array
        block_assignments[i] = block

    return block_assignments


def prepare_dataframe(df: pd.DataFrame):
    """
    Function prepares the dataframe and adds the parameters for the calculation of the omreznina
    """
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["block"] = individual_tariff_times(df["date_time"])
    df["month"] = df["date_time"].dt.month
    df["year"] = df["date_time"].dt.year
    # create a year_month column that combines year and month with a -
    df["year_month"] = df["year"].astype(str) + "-" + df["month"].astype(str)
    return df

def calculate_omreznina(df: pd.DataFrame, args):
    """
    Function calculates the omreznina.

    Parameters:
    -----------
    df: pd.DataFrame
        The dataframe that contains the follwoing parameters:
              | date_time | p |
              |     x     | x |
    """
    df = prepare_dataframe(df)
    placilo_omr_e = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    placilo_omr_p = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    placilo_omr_penali = [0, 0, 0, 0, 0]
    if args["net_metering"]:
        if df.a.sum() < 0:
            placilo_omr_e = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        else:
            placilo_omr_e = energija_omreznine(df, args["bloki_cene_energije"], placilo_omr_e)
    else:
        placilo_omr_e = energija_omreznine(df, args["bloki_cene_energije"], placilo_omr_e)

    bloki = predlagane_obr_moci(df, args["prikljucna_moc"], args["st_faz"], args["skupina_koncnih_odjemalcev"])
    placilo_omr_p = moc_omreznine(df, bloki, args["bloki_cene_moci"], placilo_omr_p)
    placilo_omr_penali = penali(df, bloki, args["Fex"], placilo_omr_penali)

    return placilo_omr_e, placilo_omr_p, placilo_omr_penali

In [4]:
path = "/Users/blazdobravec/Documents/WORK/INTERNI-PROJEKTI/02_DEV_PROJECTS/tarifni_sistem_si_web_app/data/6-113167-15minMeritve2023-01-01-2023-12-31_leto2023.xlsx"
args = {
    "bloki_cene_moci": np.array([0.24923, 0.04877, 0.01103, 0.00038, 0.00000]) + np.array([3.36401, 0.83363, 0.18034, 0.01278, 0.00000]),
    "bloki_cene_energije": np.array([0.00663, 0.00620, 0.00589, 0.00592, 0.00589]) + np.array([0.01295, 0.01224, 0.01248, 0.01246, 0.01258]),
    "prikljucna_moc": 14,
    "skupina_koncnih_odjemalcev": 0,
    "st_faz": 3,
    "net_metering": False,
    "Fex": 0.9,  # faktor presežene obračunske moči,
    "VAT": 1.22
}
time_step = 0.25

df = pd.read_excel(path, engine="openpyxl", parse_dates=["Časovna značka"])
df["p"] = df["P+ Prejeta delovna moč"] - df["P- Oddana delovna moč"]
df["q"] = df["Q+ Prejeta jalova moč"] - df["Q- Oddana jalova moč"]
df["a"] = df["p"] * time_step
df.rename(columns={"Blok": "block", "Časovna značka": "date_time"}, inplace=True)


df = df[["block", "date_time", "p", "q", "a"]]

placilo_omr_e, placilo_omr_p, placilo_omr_penali = calculate_omreznina(df)
print("leto omreznina energija:", np.sum(placilo_omr_e)*VAT)
print("leto omreznina moč:", np.sum(placilo_omr_p)*VAT)
print("leto omreznina penali:", np.sum(placilo_omr_penali)*VAT)
print("sum:", (np.sum(placilo_omr_p)*VAT + np.sum(placilo_omr_e)*VAT + np.sum(placilo_omr_penali)*VAT))

TypeError: calculate_omreznina() missing 1 required positional argument: 'args'